In [ ]:
import joblib
import pandas as pd

classic_model = joblib.load('emotion_classification_model.pkl')

def analyze_emotion_classic(text):
    emotion = classic_model.predict([text])[0]

    probabilities = {}
    if hasattr(classic_model, 'predict_proba') or hasattr(classic_model[-1], 'predict_proba'):
        proba = classic_model.predict_proba([text])[0]
        classes = classic_model[-1].classes_
        probabilities = dict(zip(classes, proba))
        
    return {
        'emotion': emotion,
        'probabilities': probabilities
    }

text = "I'm feeling really happy today because I got a promotion at work!"
result = analyze_emotion_classic(text)
print(f"Текст: '{text}'")
print(f"Определенная эмоция: {result['emotion']}")
if result['probabilities']:
    print("Вероятности эмоций:")
    for emotion, prob in sorted(result['probabilities'].items(), key=lambda x: x[1], reverse=True)[:3]:
        print(f"  - {emotion}: {prob:.4f} ({prob*100:.1f}%)")


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

bert_model_path = './bert_emotion_model/'
tokenizer = BertTokenizer.from_pretrained(bert_model_path)
model = BertForSequenceClassification.from_pretrained(bert_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

label_to_emotion = {
    "LABEL_0": "hate",         
    "LABEL_1": "neutral",      
    "LABEL_2": "anger",       
    "LABEL_3": "worry",        
    "LABEL_4": "love",         
    "LABEL_5": "empty",        
    "LABEL_6": "happiness",    
    "LABEL_7": "enthusiasm",   
    "LABEL_8": "fun",          
    "LABEL_9": "relief",       
    "LABEL_10": "sadness",     
    "LABEL_11": "surprise",    
    "LABEL_12": "relief"       
}

def analyze_emotion_bert(text):
    model.eval()

    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()[0]

    predicted_class = np.argmax(probs)

    emotion_labels = model.config.id2label
    label = emotion_labels[predicted_class]

    predicted_emotion = label_to_emotion.get(label, label)

    emotion_probabilities = {}
    for i, prob in enumerate(probs):
        label = emotion_labels[i]
        emotion = label_to_emotion.get(label, label)
        emotion_probabilities[emotion] = float(prob)
    
    return {
        'emotion': predicted_emotion,
        'probabilities': emotion_probabilities
    }

text = "I'm feeling really happy today because I got a promotion at work!"
result = analyze_emotion_bert(text)
print(f"Текст: '{text}'")
print(f"Определенная эмоция: {result['emotion']}")
print("Вероятности эмоций:")
for emotion, prob in sorted(result['probabilities'].items(), key=lambda x: x[1], reverse=True)[:3]:
    print(f"  - {emotion}: {prob:.4f} ({prob*100:.1f}%)")
